## Notebook Vb-AGP + IS : 4 branches

In [1]:
import numpy as np
import scipy as sci
from scipy.stats import norm
import openturns as ot
from gaussian_process import GaussianProcess
from scipy import stats
from NAIS_min2_ot import NAIS_m
import math
#from AK_MCS_classic import AK_MCS




import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import rcParams
from matplotlib import rc

from matplotlib import cm

params = {'backend': 'ps',
      'axes.labelsize': 55,
      'legend.fontsize': 44,
      'xtick.labelsize': 40,
      'ytick.labelsize': 40,
      'text.usetex': True,
      'text.latex.unicode':True}

rcParams.update(params)

Code algorithme Vb-AGP + MCS

In [2]:
def Vb_AGP_IS(performance_function,rv_distribution,initial_MC_sample_size = None,initial_doe_size = None, iter_max = 100, cov_max = 0.1, hot_start = False,path='',max_IS_size=50000,corr='matern52'):
    var_G_E_X_l=list()
    var_X_E_G_l=list()
    var_tot_l=list()
    var_tot=0.
    var_tot_inf=0.
    var_tot_sup=0.
    pf_l=list()
    gp_new=True
    if hot_start == False:
        #initial MC population
        MC_sample_size = initial_MC_sample_size
        MC_sample = np.array(rv_distribution.getSample(MC_sample_size))
        # initial DoE generation
        lhs = ot.LHSExperiment(rv_distribution, initial_doe_size)
        lhs.setAlwaysShuffle(True) # randomized
        spaceFilling = ot.SpaceFillingC2()
        N = 50000
        optimalLHSAlgorithm = ot.MonteCarloLHS(lhs, N, spaceFilling)
        DoE=np.array(optimalLHSAlgorithm.generate())
        
        DoE_size = DoE.shape[0]
        print("--------------------------------------")
        print("Evalulation of the perfomance function on the initial doe of size ",initial_doe_size)
        print("--------------------------------------")
#        Evaluation of the performance function
        Y = np.zeros((initial_doe_size,1))
        i = 0
        for x in DoE:
            Y[i,0] = performance_function(x)
            print(Y[i,0])
            i = i+1
        print( "--------------------------------------")
        print("Saving the initial DoE ")
        print( "--------------------------------------")
        MC_sample.dump(path+str('MC_sample_init.pk'))
        DoE.dump(path+str('DoE_init.pk'))
        Y.dump(path+str('Y_init.pk'))
        
    elif hot_start == 'init':
        MC_sample = np.load(path+str('MC_sample_init.pk'))
        MC_sample_size = MC_sample.shape[0] 
        initial_MC_sample_size =MC_sample_size 
        DoE = np.load(path+str('DoE_init.pk'))
        DoE_size = DoE.shape[0]
        Y = np.load(path+str('Y_init.pk'))
        print( "--------------------------------------")
        print ("Reading the initial DoE of size ",DoE_size)
        print( "--------------------------------------")
 

    elif hot_start == 'init_doe':
        MC_sample_size = initial_MC_sample_size
        MC_sample = rv_distribution.rvs(MC_sample_size)
    
        DoE = np.load(path+str('DoE_init.pk'),allow_pickle=True)
        DoE_size = DoE.shape[0]
        Y = np.zeros((DoE_size,1))
        i = 0
        for x in DoE:
            Y[i,0] = performance_function(x)
            print(Y[i,0])
            i = i+1
        print( "--------------------------------------")
        print ("Reading the initial DoE of size ",DoE_size)
        print( "--------------------------------------")

        
        
    elif hot_start == True:
        MC_sample = np.load(path+str('MC_sample.pk'),allow_pickle=True)
        MC_sample_size = MC_sample.shape[0]  
        initial_MC_sample_size =MC_sample_size 
        DoE = np.load(path+str('DoE.pk'),allow_pickle=True)
        DoE_size = DoE.shape[0]
        Y = np.load(path+str('Y.pk'),allow_pickle=True)
        print( "--------------------------------------")
        print ("Hot start with a DoE of size ",DoE_size)
        print( "--------------------------------------")
        
    #construction of the GP surrogate
    stochastic_dim = DoE.shape[1]    
    theta0 = np.array([0.1]*stochastic_dim)
    thetaL = np.array([1e-6]*stochastic_dim)
    thetaU = np.array([100.0]*stochastic_dim)
    gp_g = GaussianProcess(corr=corr,theta0 = theta0,thetaL=thetaL,thetaU=thetaU)     


    #AK-MCS loop
    n_iter = 0
    convergence = False
    learn_Faux=True 
    MC_sample_IS=MC_sample
    sample_IS_tot=MC_sample
    IS_sample_size=MC_sample_size
    initial_IS_sample_size=10000
    f_aux=False
    weights_IS=np.ones((IS_sample_size,1))


    while n_iter<iter_max and convergence == False:
        #fitting the GP surrogate
        gp_g.fit(DoE,Y)
        #estimation of the performance function surrogate model at the MC points
        MC_sample_G_IS = gp_g.predict(MC_sample_IS)
        


        
        #estimation of Pf on IS samples
        indicatrice = np.zeros((IS_sample_size,1))
        ind = MC_sample_G_IS<=0.0
        indicatrice[ind] = 1.0
        indicatrice=indicatrice*weights_IS
        Pf = indicatrice.sum()/IS_sample_size
        
        print( "--------------------------------------")
        print ("iteration number ",n_iter)
        print ("Pf = ",Pf)
        
        if Pf > 0. :
                
            # using the function sensitivity_analysis : computation of variances var_G_E_X (GP) et var_X_E_G (MC)
            # OR the function sensitivity_analysis_IS : computation of variances var_G_E_X (GP) et var_X_E_G (IS)
            # +  control of the stopping criterion 1st step (COV_red) : end_learning = boolean 
            if f_aux==False : # before 1st NAIS
                results_sensitivity=sensitivity_analysis_MC(gp_g,rv_distribution,MC_sample,Pf,cov_max,gp_new=gp_new,path=path)
                var_X_E_G, var_G_E_X,end_learning,var_X_E_G_inf,var_X_E_G_sup,var_G_E_X_inf,var_G_E_X_sup=results_sensitivity
            else:
                results_sensitivity=sensitivity_analysis_IS(gp_g,rv_distribution,MC_sample_IS,weights_IS,Pf,cov_max,f_aux,learn_Faux,gp_new=gp_new,path=path)
                end_learning,var_X_E_G, var_G_E_X=results_sensitivity        
                
                    
            var_G_E_X_l.append(var_G_E_X)
            var_X_E_G_l.append(var_X_E_G)
         
            np.array(var_X_E_G_l).dump('var_X_E_G_IS.pk')   
            np.array(var_G_E_X_l).dump('var_G_E_X_IS.pk')    
            
            print('cov_IS = ', np.sqrt(var_X_E_G)/Pf)
            print('cov_GP = ',np.sqrt(var_G_E_X)/Pf)
            
           
            
            pf_l.append(Pf)  
            np.array(pf_l).dump(path+'pf_algo_IS.pk')   

            
            if end_learning : 
                var_tot,var_tot_inf,var_tot_sup,Pf=total_variance_bootstrap(gp_g,rv_distribution,IS_sample_size,Pf,cov_max,MC_sample_IS,weights_IS,f_aux)
                var_target=(cov_max*Pf)**2
                
                print('COV_tot =',np.sqrt(var_tot)/Pf)
                if var_tot_sup < var_target :
                    end_learning=True
                else : 
                    end_learning = False
                
                
                    


            if end_learning :
                print('End of learning phase')
                convergence=True
                
                
            # if stopping criterion not completed        
            if end_learning == False : 
                test_var = var_X_E_G > var_G_E_X and IS_sample_size < max_IS_size
                        
                if test_var :
                    print('higher variance due to : MC/IS')   
                    if learn_Faux : 
                        gp_new=False
                        print( "UPDATE OF AUXILIARY DENSITY")
                        #NAIS 
                        IS_sample_size = 10000 # final IS population size
                        NAIS_budget= 30000 # nb of samples per iteration
                        Pf, f_aux, var_NAIS,sample_IS_tot =NAIS_m(20.,0.,rv_distribution,NAIS_budget,gp_g,iter_max=40)
                        f_aux=f_aux[-1]
                        print( "NEW IS INITIAL POPULATION")
                        MC_sample_IS=f_aux.resample(IS_sample_size).T
                        learn_Faux=False
                        t0=t.time()
                        print("WEIGHTS COMPUTATION")
                        f_aux_ISsamples=f_aux.pdf(MC_sample_IS.T)
                        distribution_ISsample=np.array(rv_distribution.computePDF(MC_sample_IS))[:,0]
                        weights_IS=distribution_ISsamples/f_aux_ISsamples
                        weights_IS=weights_IS.reshape(weights_IS.shape[0],1)
                        t1=t.time()

                        
                    else: 
                        gp_new=False
                        # + grande pop IS 
                        print( "NEW IS SAMPLES")
                        #add  MC samples
                        new_MC_sample_IS = np.zeros((MC_sample_IS.shape[0]+initial_IS_sample_size,MC_sample_IS.shape[1]))
                        new_MC_sample_IS[0:MC_sample_IS.shape[0],:] = MC_sample_IS
                        new_MC_sample_IS[MC_sample_IS.shape[0]:,:] = f_aux.resample(initial_IS_sample_size).T
                        print("WEIGHTS COMPUTATION")
                        new_weights_IS = np.zeros((MC_sample_IS.shape[0]+initial_IS_sample_size,1))
                        new_weights_IS[0:weights_IS.shape[0],:] = weights_IS
                        f_aux_ISsamples=f_aux.pdf(new_MC_sample_IS[weights_IS.shape[0]:,:].T)
                        distribution_ISsamples=np.array(rv_distribution.computePDF(new_MC_sample_IS[weights_IS.shape[0]:,:]))[:,0]
                        new_weights_IS[MC_sample_IS.shape[0]:,:] =(distribution_ISsamples/f_aux_ISsamples).reshape(initial_IS_sample_size,1)
                        
                        weights_IS=new_weights_IS.copy()
                        MC_sample_IS = new_MC_sample_IS.copy()
                        IS_sample_size = IS_sample_size + initial_IS_sample_size
                        learn_Faux=False 

                    
                    
                    
                else :
                    gp_new=True
                    print('higher variance due to : GP')
                    MC_sample_G_IS_tot,MSE_IS_tot = gp_g.predict(sample_IS_tot,eval_MSE=True)
                    MC_sample_G_MC,MSE_MC = gp_g.predict(MC_sample,eval_MSE=True)
                    MC_sample_G=np.concatenate((MC_sample_G_IS_tot,MC_sample_G_MC))
                    MSE=np.concatenate((MSE_IS_tot,MSE_MC))
                    MC_sample_tot=np.concatenate((sample_IS_tot,MC_sample))

        

            
                    epsilon = 2.0*np.sqrt(MSE)
                    X = MC_sample_G[:,0]/np.sqrt(MSE)
                    X1 = (-epsilon-MC_sample_G[:,0])/np.sqrt(MSE)
                    X2 = (epsilon-MC_sample_G[:,0])/np.sqrt(MSE)
                    EFF = MC_sample_G[:,0]*(2.0*sci.stats.norm.cdf(-X)-sci.stats.norm.cdf(X1)-sci.stats.norm.cdf(X2))-\
                    np.sqrt(MSE)*(2.0*sci.stats.norm.pdf(-X)-sci.stats.norm.pdf(X1)-sci.stats.norm.pdf(X2))+\
                    epsilon*(sci.stats.norm.cdf(X2)-sci.stats.norm.cdf(X1))
                    crit_dist = False
                    while crit_dist == False:
                        ind_max = np.argmax(EFF)
                        val_star = EFF[ind_max]
                        x_star = MC_sample_tot[ind_max]
                        #x_star not in  DoE ?
                        if np.sqrt(np.sum((DoE-x_star)**2,axis=1)).min()>=np.sqrt(np.sum((DoE-x_star)**2,axis=1)).max()/10**3:
                            crit_dist = True
                        else:
                            crit_dist = False
                            EFF[ind_max] = -10**6
                    
                    print ("x min =",x_star)
                    print ("val min=", val_star)
                    y_star = performance_function(x_star)
                    new_DoE = np.zeros((DoE_size+1,DoE.shape[1]))
                    new_DoE[0:-1,:] = DoE
                    new_DoE[-1,:] = x_star
                    new_Y = np.zeros((DoE_size+1,1))
                    new_Y[0:-1,:] = Y
                    new_Y[-1,:] = y_star
                    DoE = new_DoE.copy()
                    Y = new_Y.copy()                    
                    DoE_size = DoE_size + 1
                    learn_Faux=True
                


                    
        elif Pf == 0. :
            print('no failure point in MC')
            print( "UPDATE OF AUXILIARY DENSITY")
            NAIS_budget= 30000 # samples per iteration
            IS_sample_size = 10000 # final IS population size
            Pf, f_aux, var_NAIS,sample_IS_tot =NAIS_m(20.,0.,rv_distribution,NAIS_budget,gp_g,iter_max=30)
            f_aux=f_aux[-1]
            print( "NEW IS INITIAL POP")
            MC_sample_IS=f_aux.resample(IS_sample_size).T
            learn_Faux=False
            f_aux_ISsamples=f_aux.pdf(MC_sample_IS.T)
            distribution_ISsamples=np.array(rv_distribution.computePDF(MC_sample_IS))[:,0]
            weights_IS=distribution_ISsamples/f_aux_ISsamples
            weights_IS=weights_IS.reshape(weights_IS.shape[0],1)

            
            if Pf >= 0 :
                # add learning points for the GP 
                size = 2*stochastic_dim
                print('Enrichment of the GP')
                print('Number of learning points = ', size)
                for i in np.arange(size): 
                    MC_sample_G,MSE = gp_g.predict(sample_IS_tot,eval_MSE=True)
                    X = MC_sample_G[:,0]/np.sqrt(MSE)
                    epsilon = 2.0*np.sqrt(MSE)
                    X1 = (-epsilon-MC_sample_G[:,0])/np.sqrt(MSE)
                    X2 = (epsilon-MC_sample_G[:,0])/np.sqrt(MSE)
                    EFF = MC_sample_G[:,0]*(2.0*sci.stats.norm.cdf(-X)-sci.stats.norm.cdf(X1)-sci.stats.norm.cdf(X2))-\
                    np.sqrt(MSE)*(2.0*sci.stats.norm.pdf(-X)-sci.stats.norm.pdf(X1)-sci.stats.norm.pdf(X2))+\
                    epsilon*(sci.stats.norm.cdf(X2)-sci.stats.norm.cdf(X1))
                    ind_max = np.argmax(EFF)
                    val_star = EFF[ind_max]
                    DoE_IS=sample_IS_tot[ind_max]
                    EFF[ind_max] = -10**6
                    x=DoE_IS
                    y_star=performance_function(x)
                    print(y_star)
                    new_DoE = np.zeros((DoE_size+1,DoE.shape[1]))
                    new_DoE[0:DoE_size,:] = DoE
                    new_DoE[DoE_size:,:] = DoE_IS
                    new_Y = np.zeros((DoE_size+1,1))
                    new_Y[0:DoE_size,:] = Y
                    new_Y[DoE_size:,:] = y_star
                    DoE = new_DoE.copy()
                    Y = new_Y.copy()
                    gp_g.fit(DoE,Y)
                    DoE_size = DoE_size + 1


                    
                learn_Faux=True



                    
                
                
        n_iter = n_iter + 1
        MC_sample.dump(path+str('MC_sample.pk'))
        DoE.dump(path+str('DoE.pk'))
        Y.dump(path+str('Y.pk'))
     
    print('FIN Vb-AGP + IS')
    return Pf, DoE, n_iter, gp_g, Y, IS_sample_size, f_aux,np.sqrt(var_tot)/Pf,np.sqrt(var_tot_inf)/Pf,np.sqrt(var_tot_sup)/Pf

Code : analyse de sensibilité pour l'estimation de $V_{\tilde{X}}$ et $V_{\mathcal{G}_n}$ quand $\tilde{X}$ est une population de MC 

In [3]:
def sensitivity_analysis(gp_g,distribution,MC_sample,pf,cov_max):
    
    Omega=np.array([[-4.,4.]*stochastic_dim]).reshape(stochastic_dim,2) # domain for the discretization for KL 
    eig, vect_star, nodes, weights, M =  gp_g.K_eig_noncond_randuniform(nb_vect=1000, Omega=Omega,normalize_Omega=False,n_start=2000,eps=1e-8)
    phi = gp_g.eval_phi_KL( weights, nodes, eig, vect_star, M, MC_sample) #evaluation of the eigenfunctions at the MC points
    # setup for GP trajectories simulation for V_G_E_X
    n_traj_max=15000 
    n_traj=2000
    
    pf_samples_list=list()
    phi_n = stats.norm()
    cond = False
    end_learning = False
    
    ### Computation of var_X_E_G
    MC_sample_size=MC_sample.shape[0]
    MC_sample_G,MSE=gp_g.predict(MC_sample,eval_MSE=True)
    U=-MC_sample_G[:,0]/np.sqrt(MSE)
    # computation of p_i
    p=phi_n.cdf(U.reshape((MC_sample_size,1)))
    var_X_E_G=np.var(p)/MC_sample_size
    # computation of var_X_E_G confidence interval 
    n=MC_sample_size
    p_mean=p.mean()
    v=var_X_E_G
    v_k=np.var((p-p_mean)**2)/(n*(n-1)**2)
    var_X_E_G_inf=v-3*np.sqrt(v_k)
    var_X_E_G_sup=v+3*np.sqrt(v_k)
    
    ### Computation of var_G_E_X
    cond =False
    pf_samples=np.array([0])
    while not(cond and pf_samples.shape[0] > 3000 or (pf_samples.shape[0] > n_traj_max) and pf_samples.shape[0] > 3000) :  
        # simu trajectoires
        xi_traj=np.random.normal(0,1,(M,n_traj))
        traj,phi=gp_g.eval_KL_expansion_trajectories_phi(weights=weights,nodes=nodes,eig=eig,vect=vect_star, M=M, x_new=MC_sample,xi_traj=xi_traj,phi=phi)
        # calcul des Pf par traj 
        indicatrice = np.zeros((MC_sample_size,n_traj))
        ind = traj.T <=0.0
        indicatrice[ind] = 1.0
        pf_mu= indicatrice.sum(axis=0)/MC_sample_size
        pf_samples=pf_mu
        pf_samples_list.append(pf_samples)
        pf_samples=np.array(pf_samples_list).flatten()
        pf_tot_mean=np.mean(pf_samples)

        var_G_E_X=((pf_samples-pf_tot_mean)**2).sum()/(pf_samples.shape[0]-1)  
        print('n_traj :',pf_samples.shape[0])
        # computation of var_G_E_X confidence interval 
        n=pf_samples.shape[0]
        v=var_G_E_X
        v_k=np.var((pf_samples-pf_tot_mean)**2)*n/(n-1)**2
        var_G_E_X_inf=v-3*np.sqrt(v_k)
        var_G_E_X_sup=v+3*np.sqrt(v_k)
        # verification that the intersection of the two confidence intervals is null 
        if not((var_G_E_X_inf < var_X_E_G_inf < var_G_E_X_sup) or (var_G_E_X_inf < var_X_E_G_sup < var_G_E_X_sup)):
            cond= True
            
    # verification of the 1st stopping criterion (COV_red)
    var_target=(cov_max*pf)**2
    if (var_G_E_X_sup+ var_X_E_G_sup < var_target):
        print('COV_red = ',np.sqrt(var_G_E_X+ var_X_E_G)/pf)
        end_learning=True      
     
    
    return var_X_E_G, var_G_E_X,end_learning,var_X_E_G_inf,var_X_E_G_sup,var_G_E_X_inf,var_G_E_X_sup



Code : analyse de sensibilité pour l'estimation de $V_{\tilde{X}}$ et $V_{\mathcal{G}_n}$ quand $\tilde{X}$ est une population d'IS

In [4]:
def sensitivity_analysis_IS(gp_g,distribution,IS_sample,weights_IS,pf,cov_max,f_aux,learn_Faux,gp_new=True,path=''):
   
    stochastic_dim=gp_g.X.shape[1]
    if gp_new : 
        Omega=np.array([[-8.,8.]*stochastic_dim]).reshape(stochastic_dim,2)

        eig, vect_star, nodes, weights, M =   gp_g.K_eig_noncond_randuniform(nb_vect=1100, Omega=Omega,n_start=2000,eps=1e-8) 
        eig.dump(path+'eig.pk')
        vect_star[:,:M].dump(path+'vect_star.pk')
        nodes.dump(path+'nodes.pk')
        weights.dump(path+'weights.pk')
        np.array(M).dump(path+'M.pk')
    else : 
        eig=np.load(path+'eig.pk',allow_pickle=True)
        vect_star=np.load(path+'vect_star.pk',allow_pickle=True)
        nodes=np.load(path+'nodes.pk',allow_pickle=True)
        weights=np.load(path+'weights.pk',allow_pickle=True)
        M=int(np.load(path+'M.pk',allow_pickle=True))
        
        
    phi = gp_g.eval_phi_KL( weights, nodes, eig, vect_star, M, IS_sample)

     # setup for GP trajectories simulation for V_G_E_X
    n_traj_max=10000
    n_traj=4000
    pf_samples_list = list()
    phi_n = stats.norm()
    end_learning = False
    IS_sample_size=IS_sample.shape[0]
    
    MC_sample_G,MSE=gp_g.predict(IS_sample,eval_MSE=True)
    U=-MC_sample_G[:,0]/np.sqrt(MSE)
    # calcul des pi 
    p=phi_n.cdf(U.reshape((IS_sample_size,1)))
    var_X_E_G=np.var(p*weights_IS)/IS_sample_size
    n=IS_sample_size
    p_mean=np.mean(p*weights_IS)
    v=var_X_E_G
    # var_X_E_G confidence interval estimation 
    v_k=np.var((p*weights_IS-p_mean)**2)/(n*(n-1)**2)
    var_X_E_G_inf=v-3*np.sqrt(v_k)
    var_X_E_G_sup=v+3*np.sqrt(v_k)
    
    Pf_p=np.mean(p*weights_IS)
    cond =False
    pf_samples=np.array([0])
    while not(cond and pf_samples.shape[0] > 2000 or (pf_samples.shape[0] > n_traj_max) and pf_samples.shape[0] > 2000): 
        xi_traj=np.random.normal(0,1,(M,n_traj))
        traj,phi=gp_g.eval_KL_expansion_trajectories_phi(weights=weights,nodes=nodes,eig=eig,vect=vect_star, M=M, x_new=IS_sample,xi_traj=xi_traj,phi=phi)
        indicatrice = np.zeros((IS_sample_size,n_traj))
        ind = traj.T <=0.0
        indicatrice[ind] = 1.0
        indicatrice=indicatrice*weights_IS
        pf_mu= indicatrice.sum(axis=0)/IS_sample_size
        pf_samples=pf_mu
        pf_samples_list.append(pf_samples)
        pf_samples=np.array(pf_samples_list).flatten()
        pf_tot_mean=np.mean(pf_samples)

        var_G_E_X=((pf_samples-pf_tot_mean)**2).sum()/(pf_samples.shape[0]-1)  
        print('n_traj :',pf_samples.shape[0])
        n=pf_samples.shape[0]
        # var_G_E_X confidence interval estimation 
        v=var_G_E_X
        v_k=np.var((pf_samples-pf_tot_mean)**2)*n/(n-1)**2
        var_G_E_X_inf=v-3*np.sqrt(v_k)
        var_G_E_X_sup=v+3*np.sqrt(v_k)
        
        
        # verification that the intersection of the two confidence intervals is null 
        if not((var_G_E_X_inf < var_X_E_G_inf < var_G_E_X_sup) or (var_G_E_X_inf < var_X_E_G_sup < var_G_E_X_sup)):
            cond= True
            
    # verification of the 1st stopping criterion (COV_red)
    var_target=(cov_max*pf)**2
    if (var_G_E_X_sup+ var_X_E_G_sup < var_target):
        print('COV_red = ',np.sqrt(var_G_E_X+ var_X_E_G)/pf)
        end_learning=True

    return end_learning,var_X_E_G, var_G_E_X

Code : Estimation de $V_{tot}$ et $\hat{P}_f^t$ par Bootstrap 

In [5]:
def total_variance_bootstrap(gp_g,distribution,IS_sample_size,pf,cov_max,IS_sample,weights_IS_tot,f_aux):
    stochastic_dim=gp_g.X.shape[1]
    Omega=np.array([[-8.,8.]*stochastic_dim]).reshape(stochastic_dim,2)

    eig, vect_star, nodes, weights, M = gp_g.K_eig_noncond_randuniform(nb_vect=1100, Omega=Omega,n_start=2000,eps=1e-8)
    phi = gp_g.eval_phi_KL( weights, nodes, eig, vect_star, M, IS_sample)
    
    # number of realizations of MC and GP 
    N_mc_i=200 
    N_mc_max=5000 
    n_traj=1
    pf_list=list()
    var_target=(cov_max*pf)**2
    var_tot_inf=var_target-1
    var_tot_sup=var_target+1
    N_mc=N_mc_i
    traj_ind=np.arange(IS_sample_size)

    
    while not( not(var_tot_inf<var_target<var_tot_sup) or N_mc>N_mc_max) :
        n_traj=N_mc_i
        xi_traj=np.random.normal(0,1,(M,n_traj))
        traj_i,phi=gp_g.eval_KL_expansion_trajectories_phi(weights=weights,nodes=nodes,eig=eig,vect=vect_star, M=M, x_new=IS_sample,xi_traj=xi_traj,phi=phi) 
        for i in np.arange(N_mc_i):
            ind_boot=np.random.choice(traj_ind,size=IS_sample_size,replace=True)
            traj=traj_i.T[ind_boot,i]
            weights_IS=weights_IS_tot[ind_boot]
            indicatrice = np.zeros((IS_sample_size,n_traj))
            ind = traj <=0.0
            indicatrice[ind] = 1.0
            indicatrice=indicatrice*weights_IS
            Pf = indicatrice.sum(axis=0)/IS_sample_size
            pf_list.append(Pf)
        
        pf=np.array(pf_list).flatten()
        pf_tot_mean=np.mean(pf)
        var_tot=((pf-pf_tot_mean)**2).sum()/(pf.shape[0]-1)
        # total variance confidence interval computation
        n=pf.shape[0]
        v=var_tot
        v_k=np.var((pf-pf_tot_mean)**2)*n/(n-1)**2
        var_tot_inf=v-3*np.sqrt(v_k)
        var_tot_sup=v+3*np.sqrt(v_k)
        N_mc=N_mc+N_mc_i 

    
    return var_tot,var_tot_inf,var_tot_sup,pf_tot_mean

In [6]:
def f_4b(X):
    U1=3+0.1*((X[0]-X[1])**2)-((X[0]+X[1])/np.sqrt(2))
    U2=3+0.1*((X[0]-X[1])**2)+((X[0]+X[1])/np.sqrt(2))
     
    U3=X[0]-X[1]+(6./np.sqrt(2))
    U4=X[1]-X[0]+(6./np.sqrt(2))
#    U5=X[:,0]-X[:,1]+(7/np.sqrt(2))*np.ones(n).reshape(n,1)
#    U6=X[:,1]-X[:,0]+(7/np.sqrt(2))*np.ones(n).reshape(n,1)
    Z=np.minimum(U1,U2)
    Z=np.minimum(Z,np.array(U3))
    Z=np.minimum(Z,np.array(U4))
#    Z=np.minimum(Z,np.array(U5))
#    Z=np.minimum(Z,np.array(U6))
    return Z +1.5
stochastic_dim = 2
distribution_4b=ot.Normal(ot.Point([0]*stochastic_dim), ot.CovarianceMatrix(stochastic_dim))

In [7]:
###########  choix cas test + distribution et param algo 
fct_test= f_4b
distrib_test=distribution_4b
initial_doe_size=16
initial_MC_sample_size=10000
cov_max=0.1

In [8]:
Pf,DoE,n_iter,gp_g, Y, IS_sample_size,f_aux, cov_tot, cov_tot_inf,cov_tot_sup= Vb_AGP_IS(fct_test,distrib_test,initial_MC_sample_size = initial_MC_sample_size,initial_doe_size = initial_doe_size,  iter_max = 200, cov_max=cov_max,corr='matern52')


--------------------------------------
('Evalulation of the perfomance function on the initial doe of size ', 16)
--------------------------------------
4.600553583340887
3.0605503072076674
2.751161753000482
3.825448710035188
4.128991614244523
4.173599874735466
3.3942990188114295
2.89011217334915
3.8668445505702076
4.070479588506203
4.049800215120483
4.01047638687291
3.8575737443092875
2.7676458127265793
2.8942634844567414
4.416545016676093
--------------------------------------
Saving the initial DoE 
--------------------------------------
--------------------------------------
('iteration number ', 0)
('Pf = ', 0.0)
no failure point in MC
UPDATE OF AUXILIARY DENSITY
2.8190456505784978
2.7756161907083494
NEW IS INITIAL POP
Enrichment of the GP
('Number of learning points = ', 4L)
0.0914372542878068
0.4038469743476343
-0.1979259506891351
0.31900777240641265
--------------------------------------
('iteration number ', 1)
('Pf = ', 3.408852905672263e-05)
('n_traj :', 4000L)
('cov_IS = ',

KeyboardInterrupt: 